### IMPORTS

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
root = os.environ.get("ROOT_PATH", ".")
os.chdir(root)
from datetime import datetime, timedelta
from data_preprocessing.fetch_data import fetch_ohlcv_to_df
from data_preprocessing.training_helpers import evaluate_token_strategy
from data_preprocessing.training_helpers import create_merged_feature
import tensorflow as tf
from data_preprocessing.training_helpers import create_all_features
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.utils import to_categorical
import warnings
from sklearn.decomposition import PCA
from scikeras.wrappers import KerasClassifier
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping

warnings.filterwarnings("ignore")

### CONSTATNTS

In [ ]:
EXCHANGE = "binance"
TRAIN_START_DATE = "2024-07-01 00:00:00"
TRAIN_END_DATE = "2025-02-15 00:01:00"
SET_OVERLAPPING_DAYS = 20
VAL_START_DATE = (datetime.strptime(TRAIN_END_DATE, "%Y-%m-%d %H:%M:%S") - timedelta(days=SET_OVERLAPPING_DAYS)).strftime("%Y-%m-%d %H:%M:%S")
VAL_END_DATE = "2025-03-15 00:01:00"
TEST_START_DATE = (datetime.strptime(VAL_END_DATE, "%Y-%m-%d %H:%M:%S") - timedelta(days=SET_OVERLAPPING_DAYS)).strftime("%Y-%m-%d %H:%M:%S")
TEST_END_DATE = "2025-06-10 00:01:00"
ALL_START_DATE = "2024-10-20 00:00:00"
ALL_END_DATE = "2025-06-10 00:01:00"
LONG_PROBA = 0.6
SHORT_PROBA = 0.6
TAKE_PROFIT = 2
STOP_LOSS = 0.8
TAKE_PROFIT_PCT = TAKE_PROFIT/100
STOP_LOSS_PCT = STOP_LOSS / 100
CLASS_WEIGHT = { 0:0.3, 1: 1, 2:1 }

### LOAD DATA

In [ ]:
df_train_ada = await fetch_ohlcv_to_df("ADA/USDT", TRAIN_START_DATE, TRAIN_END_DATE)
df_val_ada = await fetch_ohlcv_to_df("ADA/USDT", VAL_START_DATE, VAL_END_DATE)
df_test_ada = await fetch_ohlcv_to_df("ADA/USDT", TEST_START_DATE, TEST_END_DATE)

df_train_btc = await fetch_ohlcv_to_df("BTC/USDT", TRAIN_START_DATE, TRAIN_END_DATE)
df_val_btc = await fetch_ohlcv_to_df("BTC/USDT", VAL_START_DATE, VAL_END_DATE)
df_test_btc = await fetch_ohlcv_to_df("BTC/USDT", TEST_START_DATE, TEST_END_DATE)

### PREPROCESS DATA

In [ ]:
train_data_btc = create_all_features(df=df_train_btc, token="btc", target_type="pct", tp=2, sl=0.8, max_bars=60*6, atr_period=60*6)
val_data_btc = create_all_features(df=df_val_btc, token="btc", target_type="pct", tp=2, sl=0.8, max_bars=60*6, atr_period=60*6)
test_data_btc = create_all_features(df=df_test_btc, token="btc", target_type="pct", tp=2, sl=0.8, max_bars=60*6, atr_period=60*6)

train_data_ada = create_all_features(df=df_train_ada, token="ada", target_type="pct", tp=2, sl=0.8, max_bars=60*6, atr_period=60*6)
val_data_ada = create_all_features(df=df_val_ada, token="ada", target_type="pct", tp=2, sl=0.8, max_bars=60*6, atr_period=60*6)
test_data_ada = create_all_features(df=df_test_ada, token="ada", target_type="pct", tp=2, sl=0.8, max_bars=60*6, atr_period=60*6)

In [ ]:
train_data = create_merged_feature(
    main_feature_df=df_train_ada,
    complementry_data_df=df_train_btc,
    main_token_name="ada",
    complementry_token_name="btc",
    target_type="pct",
    tp=2,
    sl=0.8,
    max_bars=60 * 6,
    atr_period=60 * 6,
)

val_data = create_merged_feature(
    main_feature_df=df_val_ada,
    complementry_data_df=df_val_btc,
    main_token_name="ada",
    complementry_token_name="btc",
    target_type="pct",
    tp=2,
    sl=0.8,
    max_bars=60 * 6,
    atr_period=60 * 6,
)

test_data = create_merged_feature(
    main_feature_df=df_test_ada,
    complementry_data_df=df_test_btc,
    main_token_name="ada",
    complementry_token_name="btc",
    target_type="pct",
    tp=2,
    sl=0.8,
    max_bars=60 * 6,
    atr_period=60 * 6,
)

In [ ]:
target_y = "y_combined_1h_2h_4h"
target_x = "X_combined_1h_2h_4h"

### DEFINE MODEL

In [ ]:
def get_model(input_shape, num_classes=3):
    inputs = layers.Input(shape=input_shape, name="features")
    x = layers.BatchNormalization(name="batchnorm_input")(inputs)
    x = layers.Dense(
        128, activation="relu", kernel_regularizer=regularizers.l2(1e-4), name="dense_1"
    )(x)
    x = layers.Dropout(0.3, name="dropout_1")(x)
    x = layers.Dense(
        64, activation="relu", kernel_regularizer=regularizers.l2(1e-4), name="dense_2"
    )(x)
    x = layers.Dropout(0.3, name="dropout_2")(x)
    x = layers.Dense(
        32, activation="relu", kernel_regularizer=regularizers.l2(1e-4), name="dense_3"
    )(x)
    x = layers.Dropout(0.2, name="dropout_3")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="output")(x)
    model = models.Model(inputs=inputs, outputs=outputs, name="mlp_3class")
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.003),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [ ]:
def create_model():
    return get_model(input_shape=(250,), num_classes=3)

clf = KerasClassifier(
    model=create_model,
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["precision"],
    epochs=50,
    batch_size=32,
    verbose=0,
)

nn_pipeline = Pipeline([
    ('pca', PCA(n_components=250)),
    ('clf', clf),
])

nn_ada = get_model(input_shape=(train_data_btc[target_x].shape[1],), num_classes=3)
nn_btc = get_model(input_shape=(train_data_btc[target_x].shape[1],), num_classes=3)

es = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)


### TRAIN MODEL ( EXPERIMENT )

In [ ]:
nn_pipeline.fit(
    train_data[target_x],
    train_data[target_y],
)
nn_pipeline.score(val_data[target_x], val_data[target_y])

In [ ]:
X_train = train_data_btc[target_x].values
y_int = train_data_btc[target_y].values.reshape(-1)
y_ohe = to_categorical(y_int, num_classes=3)

nn_btc.fit(
    X_train,
    y_ohe,
    epochs=25,
    batch_size=32,
    verbose=0,
    callbacks=[es],
    class_weight=CLASS_WEIGHT,
)

y_int_val = val_data_btc[target_y].values.reshape(-1)
y_ohe_val = to_categorical(y_int_val, num_classes=3)
print(nn_btc.evaluate(val_data_btc[target_x], y_ohe_val))

y_int_test = test_data_btc[target_y].values.reshape(-1)
y_ohe_test = to_categorical(y_int_test, num_classes=3)
print(nn_btc.evaluate(test_data_btc[target_x], y_ohe_test))

In [ ]:
X_train = train_data_ada[target_x].values
y_int = train_data_ada[target_y].values.reshape(-1)
y_ohe = to_categorical(y_int, num_classes=3)

nn_ada.fit(
    X_train,
    y_ohe,
    epochs=25,
    batch_size=32,
    verbose=0,
    callbacks=[es],
    class_weight=CLASS_WEIGHT,
)

y_int_val = val_data_ada[target_y].values.reshape(-1)
y_ohe_val = to_categorical(y_int_val, num_classes=3)
print(nn_ada.evaluate(val_data_ada[target_x], y_ohe_val))

y_int_test = test_data_ada[target_y].values.reshape(-1)
y_ohe_test = to_categorical(y_int_test, num_classes=3)
print(nn_ada.evaluate(test_data_ada[target_x], y_ohe_test))

### EVALUATION

##### GROUPED FEATURE MODEL

In [ ]:
break_even_precision = 1/(1+(TAKE_PROFIT/STOP_LOSS))
print(f"MINIMUM PRECISION TO BREAK EVEN (BEFORE FEES) - {break_even_precision}")

In [ ]:
position_val_backtest, vectorbt_val_backtest = evaluate_token_strategy(
    model=nn_pipeline,
    data=val_data,
    data_1m=df_val_ada,
    target_x=target_x,
    target_y=target_y,
    val_end_date=TRAIN_END_DATE,
    token_name="ADA",
    classes=(1,2),
    long_proba=LONG_PROBA,
    short_proba=SHORT_PROBA,
    trade_size=1,
    use_proba=True,
    atr_target=False,
    tp = TAKE_PROFIT,
    sl = STOP_LOSS,
    atr_period = 60 * 5,
    tp_pct = TAKE_PROFIT_PCT,
    sl_pct = STOP_LOSS_PCT,
    nn = False
)

overall_winrate = position_val_backtest['metrics']['overall']['trade_metrics']['win_rate']
long_win_rate = position_val_backtest['metrics']['long']['trade_metrics']['win_rate']
short_win_rate = position_val_backtest['metrics']['short']['trade_metrics']['win_rate']

print(f"""
overall winrate => {overall_winrate}
long winrate    => {long_win_rate}
short winrate    => {short_win_rate}
""")

vectorbt_val_backtest.stats()

In [ ]:
position_test_backtest, vectorbt_test_backtest = evaluate_token_strategy(
    model=nn_pipeline,
    data=test_data,
    data_1m=df_test_ada,
    target_x=target_x,
    target_y=target_y,
    val_end_date=VAL_END_DATE,
    token_name="ADA",
    classes=(1,2),
    long_proba=LONG_PROBA,
    short_proba=SHORT_PROBA,
    trade_size=1,
    use_proba=True,
    atr_target=False,
    tp = TAKE_PROFIT,
    sl = STOP_LOSS,
    atr_period = 60 * 5,
    tp_pct = TAKE_PROFIT_PCT,
    sl_pct = STOP_LOSS_PCT,
    nn = False
)

overall_winrate = position_test_backtest['metrics']['overall']['trade_metrics']['win_rate']
long_win_rate = position_test_backtest['metrics']['long']['trade_metrics']['win_rate']
short_win_rate = position_test_backtest['metrics']['short']['trade_metrics']['win_rate']
print(f"""
overall winrate => {overall_winrate}
long winrate    => {long_win_rate}
short winrate    => {short_win_rate}
""")

vectorbt_test_backtest.stats()

##### INDIVIDUAL FEATURE MODEL

In [ ]:
position_val_backtest, vectorbt_val_backtest = evaluate_token_strategy(
    model=nn_ada,
    data=val_data_ada,
    data_1m=df_val_ada,
    target_x=target_x,
    target_y=target_y,
    val_end_date=TRAIN_END_DATE,
    token_name="ADA",
    classes=(1,2),
    long_proba=LONG_PROBA,
    short_proba=SHORT_PROBA,
    trade_size=1,
    use_proba=True,
    atr_target=False,
    tp = TAKE_PROFIT,
    sl = STOP_LOSS,
    atr_period = 60 * 5,
    tp_pct = TAKE_PROFIT_PCT,
    sl_pct = STOP_LOSS_PCT,
    nn = True
)

overall_winrate = position_val_backtest['metrics']['overall']['trade_metrics']['win_rate']
long_win_rate = position_val_backtest['metrics']['long']['trade_metrics']['win_rate']
short_win_rate = position_val_backtest['metrics']['short']['trade_metrics']['win_rate']

print(f"""
overall winrate => {overall_winrate}
long winrate    => {long_win_rate}
short winrate    => {short_win_rate}
""")

vectorbt_val_backtest.stats()

In [ ]:
position_test_backtest, vectorbt_test_backtest = evaluate_token_strategy(
    model=nn_ada,
    data=test_data_ada,
    data_1m=df_test_ada,
    target_x=target_x,
    target_y=target_y,
    val_end_date=VAL_END_DATE,
    token_name="ADA",
    classes=(1,2),
    long_proba=LONG_PROBA,
    short_proba=SHORT_PROBA,
    trade_size=1,
    use_proba=True,
    atr_target=False,
    tp = TAKE_PROFIT,
    sl = STOP_LOSS,
    atr_period = 60 * 5,
    tp_pct = TAKE_PROFIT_PCT,
    sl_pct = STOP_LOSS_PCT,
    nn = True
)

overall_winrate = position_test_backtest['metrics']['overall']['trade_metrics']['win_rate']
long_win_rate = position_test_backtest['metrics']['long']['trade_metrics']['win_rate']
short_win_rate = position_test_backtest['metrics']['short']['trade_metrics']['win_rate']
print(f"""
overall winrate => {overall_winrate}
long winrate    => {long_win_rate}
short winrate    => {short_win_rate}
""")

vectorbt_test_backtest.stats()